In [1]:
import numpy as np
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)]
)

In [2]:
# import sys

# base_path = '/tf/eipi10/qbz95'
# sys.path.append(base_path)

# from qbz95 import config
# import pandas as pd
# config = config.get_config('jupyter')
# pd.set_option('display.width', 1000)

# %matplotlib inline

# # 代码自动重新加载
# %load_ext autoreload
# %autoreload 2

# # #当module有新的方法的时候，需要运行下面方法。
# # %reload_ext autoreload


In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers, optimizers, losses

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)]
)

def get_block(x, filters, conv_count, kernel_size=(3, 3), padding='same', use_bn=True, 
              use_dropout=True, drop_out_rate=0.3):
    for i in range(conv_count):
        x = layers.Conv2D(filters, kernel_size, padding=padding, activation='relu')(x)
    if use_bn: x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    if use_dropout: x = layers.Dropout(drop_out_rate)(x)
    return x    

def get_model(input_shape, learning_rate=0.001, use_bn=True, use_dropout=True, drop_out_rate=0.3):   
    input = layers.Input(shape=input_shape)
    
    x = get_block(input, 64, conv_count=2, use_bn=use_bn, use_dropout=use_dropout, drop_out_rate=drop_out_rate)
        
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    if use_dropout: x = layers.Dropout(drop_out_rate)(x)  
 
    x = layers.Dense(10)(x)
    
    model = Model(inputs=input, outputs=x, name='vgg') 
    model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate),
                  loss=losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])      
    model.checkpoint_path = './checkpoints/{}/checkpoint'.format(model.name)
    return model   

# 创建模型
use_bn = True
use_dropout = True
drop_out_rate = 0.3
model = get_model((32, 32, 1), use_bn=use_bn, use_dropout=use_dropout, drop_out_rate=drop_out_rate)
model.summary()

Model: "vgg"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 16384)             0       

In [13]:
def get_weight_num(obj, trainable=True):
    '''得到模型可训练参数的个数'''
    if trainable:
        return int(np.sum([np.prod(p.shape) for p in obj.trainable_weights]))
    else:
        return int(np.sum([np.prod(p.shape) for p in obj.non_trainable_weights]))

def show_model(model):
    print('='*120)
    if model.name is not None:
        print('Model Name: ', model.name)       
        print('-'*120)
    trainable_weights =  get_weight_num(model)   
    non_trainable_weights =  get_weight_num(model, False)  
    total_weights = trainable_weights + non_trainable_weights
    print('total_weights: {}'.format(total_weights))
    print('trainable_weights: {}'.format(trainable_weights))
    print('non_trainable_weights: {}'.format(non_trainable_weights))  
    print('='*120)
    for i, layer in enumerate(model.layers):
        if i>0: print('-'*120) 
        if isinstance(layer, tf.python.keras.layers.convolutional.Conv2D):
            print("Conv2D(filters={}, kernel_size={}, padding={}, activation={}, output_shape={},\n       trainable_weights={}, non_trainable_weights={})".format(
                layer.filters, layer.kernel_size, layer.padding, layer.activation.__name__, layer.output.shape, get_weight_num(layer), get_weight_num(layer, False)))
        elif isinstance(layer, tf.python.keras.engine.input_layer.InputLayer):
            print("InputLayer(input_shape={}, batch_size={})".format(
                layer.input_shape, layer.batch_size))  
        elif isinstance(layer, tf.python.keras.layers.pooling.MaxPooling2D):
            print("MaxPooling2D(pool_size={}, padding={}, output_shape={})".format(
                layer.pool_size, layer.strides, layer.padding, layer.output.shape)) 
        elif isinstance(layer, tf.python.keras.layers.core.Dense):
            print("Dense(units={}, activation={}, output_shape={}, trainable_weights={}, non_trainable_weights={})".format(
                layer.units, layer.activation.__name__, layer.output.shape, get_weight_num(layer), get_weight_num(layer, False)))  
        elif isinstance(layer, tf.python.keras.layers.core.Flatten):
            print("Flatten(output_shape={})".format(layer.output.shape))        
        elif isinstance(layer, tf.python.keras.layers.core.Dropout ):
            print("Dropout(rate={}, noise_shape={}, output_shape={})".format(layer.rate, layer.noise_shape, layer.output.shape))     
        elif isinstance(layer, tf.python.keras.layers.normalization_v2.BatchNormalization ):
            print("BatchNormalization(axis={}, momentum={}, epsilon={}, center={}, scale={},\n                   trainable_weights={}, non_trainable_weights={})".format(
                layer.axis, layer.momentum, layer.epsilon, layer.center, layer.scale, get_weight_num(layer), get_weight_num(layer, False)))            
        else:
            print(layer)           
    print('='*120)   
show_model(model)

Model Name:  vgg
------------------------------------------------------------------------------------------------------------------------
total_weights: 2136394
trainable_weights: 2136266
non_trainable_weights: 128
InputLayer(input_shape=[(None, 32, 32, 1)], batch_size=None)
------------------------------------------------------------------------------------------------------------------------
Conv2D(filters=64, kernel_size=(3, 3), padding=same, activation=relu, output_shape=(None, 32, 32, 64),
       trainable_weights=640, non_trainable_weights=0)
------------------------------------------------------------------------------------------------------------------------
Conv2D(filters=64, kernel_size=(3, 3), padding=same, activation=relu, output_shape=(None, 32, 32, 64),
       trainable_weights=36928, non_trainable_weights=0)
------------------------------------------------------------------------------------------------------------------------
BatchNormalization(axis=ListWrapper([3]), m

Model: "vgg9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0      

In [30]:
import tensorflow as tf
import numpy as np

train_data=np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

def test_reshuffle_each_iteration(reshuffle_each_iteration):
    print('='*25, 'reshuffle_each_iteration={}'.format(reshuffle_each_iteration), '='*25)   
    train_val_dataset = tf.data.Dataset.from_tensor_slices(train_data) 
    train_val_dataset = train_val_dataset.shuffle(100, reshuffle_each_iteration = reshuffle_each_iteration)

    train_size = int(0.5*train_data.shape[0])
    train_dataset = train_val_dataset.take(train_size).shuffle(100, reshuffle_each_iteration = True).batch(5) 
    val_dataset = train_val_dataset.skip(train_size).shuffle(100, reshuffle_each_iteration = True).batch(5) 

    print('-'*25, 'first', '-'*25) 
    for data in train_dataset:
        print('train_dataset:', data.numpy())
       
    for data in val_dataset:
        print('val_dataset:', data.numpy())    

    print('-'*25, 'second', '-'*25)     
    for data in train_dataset:
        print('train_dataset:', data.numpy())

    for data in val_dataset:
        print('val_dataset:', data.numpy())      

test_reshuffle_each_iteration(True)

test_reshuffle_each_iteration(False)

========================= reshuffle_each_iteration=True =========================
------------------------- first -------------------------
train_dataset: [6 4 8 7 5]
val_dataset: [6 5 4 7 1]
------------------------- second -------------------------
train_dataset: [8 3 2 1 9]
val_dataset: [6 3 7 8 2]
========================= reshuffle_each_iteration=False =========================
------------------------- first -------------------------
train_dataset: [8 4 5 9 2]
val_dataset: [ 7 10  1  3  6]
------------------------- second -------------------------
train_dataset: [5 2 8 4 9]
val_dataset: [ 1  3  6 10  7]


In [50]:
import os
import shutil

def show_tree(path, max_depth=10, max_num=100):
    def _show_tree(path, depth, max_num, prefix):
        if max_num<=0 or depth>max_depth:
            return max_num
        if depth==1: 
            print(path)
            max_num=max_num-1
        items = os.listdir(path)
        for i, item in enumerate(items):
            if max_num<=0: return max_num
            newitem = path +'/'+ item
            if i==len(items)-1:
                print(prefix  + "└──" + item)            
                new_prefix = prefix+"    "                
            else:
                print(prefix  + "├──" + item)
                new_prefix = prefix+"│   "
            max_num=max_num-1
            if os.path.isdir(newitem):
                max_num = _show_tree(newitem, depth=depth+1, max_num=max_num, prefix=new_prefix)         
        return max_num
    _show_tree(path, depth=1, max_num=max_num, prefix="")
        
def create_file(file_path):
    file = open(file_path,'a')
    file.write(file_path)
    file.close()
                                
def prepare(base_path):
    
    if os.path.exists(base_path):
        shutil.rmtree(base_path)
    os.makedirs(base_path)
    create_file(os.path.join(base_path, "file01.txt"))
    create_file(os.path.join(base_path, "file02.txt"))
    
    folder1 = os.path.join(base_path, "folder1")
    folder2 = os.path.join(base_path, "folder2")
    folder3 = os.path.join(base_path, "folder3")
    
    os.makedirs(folder1)
    os.makedirs(folder2)
    os.makedirs(folder3)
    
    create_file(os.path.join(folder1, "file11.txt"))
    
    create_file(os.path.join(folder2, "file21.txt"))
    create_file(os.path.join(folder2, "file22.txt"))    
    
    folder30 = os.path.join(folder3, "folder30")
    os.makedirs(folder30)
    
    create_file(os.path.join(folder30, "file301.txt"))      
    create_file(os.path.join(folder30, "file302.txt"))     
    
    create_file(os.path.join(folder3, "file31.txt"))
    create_file(os.path.join(folder3, "file32.txt"))      
    create_file(os.path.join(folder3, "file33.txt")) 
    
    folder34 = os.path.join(folder3, "folder34")
    os.makedirs(folder34)
    
    create_file(os.path.join(folder34, "file341.txt"))      
    create_file(os.path.join(folder34, "file342.txt"))     

    
base_path = 'tmp'   
prepare(base_path)

print('-'*50)
print('显示目录树')
show_tree(base_path) 

print('-'*50)
print('显示最多11个文件或目录')
show_tree(base_path, max_num=11)

print('-'*50)
print('删除一个目录和一个文件')
shutil.rmtree(os.path.join(base_path, "folder2"))
os.remove(os.path.join(base_path, "folder3/file32.txt"))
show_tree(base_path) 

print('-'*50)
print('显示两层目录树')
show_tree(base_path, max_depth=2)




--------------------------------------------------
显示目录树
tmp
├──file01.txt
├──file02.txt
├──folder1
│   └──file11.txt
├──folder2
│   ├──file21.txt
│   └──file22.txt
└──folder3
    ├──folder30
    │   ├──file301.txt
    │   └──file302.txt
    ├──file31.txt
    ├──file32.txt
    ├──file33.txt
    └──folder34
        ├──file341.txt
        └──file342.txt
--------------------------------------------------
显示最多11个文件或目录
tmp
├──file01.txt
├──file02.txt
├──folder1
│   └──file11.txt
├──folder2
│   ├──file21.txt
│   └──file22.txt
└──folder3
    ├──folder30
    │   ├──file301.txt
--------------------------------------------------
删除一个目录和一个文件
tmp
├──file01.txt
├──file02.txt
├──folder1
│   └──file11.txt
└──folder3
    ├──folder30
    │   ├──file301.txt
    │   └──file302.txt
    ├──file31.txt
    ├──file33.txt
    └──folder34
        ├──file341.txt
        └──file342.txt
--------------------------------------------------
显示两层目录树
tmp
├──file01.txt
├──file02.txt
├──folder1
│   └──file11.txt
└──folder